## First we need to call all of the libraries we need to perform in our metadata wrangle

In [1]:
import pandas as pd
pd.options.display.width = 180
#import os
from os import walk
import shutil
import ipywidgets as widgets
from ipywidgets import *
import requests
import csv
import io

### Now let's select some metadata. 

If you have prepared metadata\* on your computer that you want to add, it is possible to upload into the repository locally using the [Add Metadata](00AddMetadata.ipynb) Notebook before completing the following cells in this notebook. Otherwise, follow along and use some of the sample metadata the following steps will help you to select.

\* Prepared metadata contains a root element that has a standardized namespace and namespace prefix. Many dialects such as ISO and DIF are consistently written this way, but some dialects such as CSDGM are often written by organizations as only well-formed XML.

Create a list of subdirectories in the collection directory of MILE2 to select metadata for evaluation

In [2]:
Organizations = []
for (dirpath, dirnames, filenames) in walk('../collection/'):
    Organizations.extend(dirnames)
    break  

Create a function to select the organization the metadata comes from

In [3]:
def OrganizationChoices(organization):
    global OrganizationChoice
    global Organization
    Organization=organization
    print("Organization of the collection is", Organization)


Create a dropdown using the Organizations list and the organization selector function. This sets the Organization variable.

In [4]:
interactive(OrganizationChoices, organization=Organizations)

Create a list of collections in the organization directory selected in the dropdown above

In [5]:
Collections = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization)):
    Collections.extend(dirnames)
    break 
Collections

['LTER',
 'CDL',
 'US_MPC',
 'LTER_EUROPE',
 'USANPN',
 'PISCO',
 'KUBI',
 'IARC',
 'RGD',
 'SEAD',
 'ONEShare',
 'GOA',
 'EDACGSTORE',
 'KNB',
 'TFRI',
 'ESA',
 'USGSCSAS',
 'GLEON',
 'DRYAD',
 'SANPARKS',
 'TERN',
 'EDORA',
 'NMEPSCOR',
 'ORNLDAAC',
 'IOE']

Create a function to select the collection the metadata comes from

In [6]:
def CollectionChoices(collection):
    global CollectionChoice
    global Collection
    Collection=collection

Create a dropdown using the Collections list and the organization selector function. This sets the Collection variable.

In [7]:
interactive(CollectionChoices, collection=Collections)

Many organizations support multiple metadata dialects, and share their collections in more than one dialect. This list is created the same way the others are. It adds the different dialects the collection is shared in to a list.

In [9]:
Dialects = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization,Collection)):
    Dialects.extend(dirnames)
    break 
dialectList=Dialects


Create a function to select the dialect you want to send to the evaluator service.

In [10]:
def dialectChoice(dialect):
    global Dialect
    Dialect=dialect
    print("Dialect of the collection is", Dialect)


Create a dropdown using the Dialects list and the dialect selector function. This sets the Dialect variable.

In [11]:
interactive(dialectChoice,dialect=dialectList)

change to the zip directory 

In [12]:
cd ../zip

/Users/scgordon/MILE2/zip


Combine the Organization, Collection, and Dialect variables with the string 'xml' as a relative path and save the string to a variable

In [13]:
MetadataDestination=os.path.join(Organization,Collection,Dialect,'xml')
MetadataDestination

'DataONE/LTER_EUROPE/EML/xml'

Use the path to create a directory structure in the zip directory

In [14]:
os.makedirs(MetadataDestination, exist_ok=True)

Create a path to the metadata you selected earlier and save the string to a variable, 'MetadataLocation'.

In [15]:
MetadataLocation=os.path.join('../collection/',Organization,Collection,Dialect,'xml')

MetadataLocation

'../collection/DataONE/LTER_EUROPE/EML/xml'

Copy the metadata to the new directory structure.

In [16]:
src_files = os.listdir(MetadataLocation)
for file_name in src_files:
    full_file_name = os.path.join(MetadataLocation, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, MetadataDestination)

Make a zip file to upload to the evaluator service

In [17]:
shutil.make_archive('../upload/metadata', 'zip', os.getcwd())

'/Users/scgordon/MILE2/upload/metadata.zip'

In [18]:
%cd ../upload 

/Users/scgordon/MILE2/upload


Send metadata to the Evaluator. Get the responses with csv encoding. This step can take up to a minute and doesn't track progress, but a dataframe or an error message will be returned.

Save the dataframe as a csv for further analysis. Copy the csv to a directory, named for the organization that had the metadata in it's holdings. Give it a filename matching the the metadata collection and dialect.

Clear up temporary files and directories, switch to the data directory

In [19]:
# Send metadata package, read the response into a dataframe
url = 'http://metadig.nceas.ucsb.edu/metadata/evaluator'
files = {'zipxml': open('metadata.zip', 'rb')}
r = requests.post(url, files=files, headers={"Accept-Encoding": "gzip"})
r.raise_for_status()
EvaluatedMetadataDF = pd.read_csv(io.StringIO(r.text), quotechar='"')

#build filepaths, directories and file names
Filedirectory=os.path.join('../data/',Organization)
os.makedirs(Filedirectory, exist_ok=True)
Filename='/'+Collection+'_'+Dialect+'_Evaluated.csv.gz'
SimplfiedFilename='/'+Collection+'_'+Dialect+'_SimplifiedEvaluated.csv.gz'
FilePath=Filedirectory+Filename
SimplifiedFilePath=Filedirectory+SimplfiedFilename
EvaluatedMetadataDF.insert(3, 'Collection', Organization+'_'+Collection+'_'+Dialect)
EvaluatedMetadataDF.to_csv(FilePath, mode = 'w', compression='gzip', index=False)

#Change directories, delete upload directory and zip. Delete copied metadata.
%cd ../
shutil.rmtree('upload')
%cd zip
shutil.rmtree(Organization)
%cd ../data

#Create a simplified XPath output
EvaluatedSimplifiedMetadataDF = EvaluatedMetadataDF.copy()
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/gco:CharacterString', '')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/[a-z]+:+?', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('/[A-Z]+_[A-Za-z]+/?', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.replace('//', '/')
EvaluatedSimplifiedMetadataDF['XPath']=EvaluatedSimplifiedMetadataDF['XPath'].str.rstrip('//')
EvaluatedSimplifiedMetadataDF.to_csv(SimplifiedFilePath, mode = 'w', compression='gzip', index=False)

/Users/scgordon/MILE2
/Users/scgordon/MILE2/zip
/Users/scgordon/MILE2/data


In [20]:
EvaluatedMetadataDF

,Concept,Content,Record,Collection,XPath
0,Abstract,These are measurements of Active Layer Thickne...,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/descript/abstract
1,Acknowledgement,Susan Hubbard,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/datacred
2,Author,John Peterson (JEPeterson@lbl.gov),ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/citation/citeinfo/origin
3,Author / Originator,John Peterson (JEPeterson@lbl.gov),ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/citation/citeinfo/origin
4,Bounding Box,-156.670518 -156.401514 71.324601 71.260956,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/spdom/bounding
5,Contributor Name,Susan Hubbard,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/datacred
6,Distribution Contact,Data Center Support Next-Generation Ecosystem ...,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/distinfo/distrib/cntinfo
7,Easternmost Longitude,-156.401514,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/spdom/bounding/eastbc
8,Keyword,moisture content of soil layer,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/keywords/theme/themekey
9,Keyword,Intensive Site 1,ftp___ngee.ornl.gov_data_outgoing_metadata_Act...,DataONE_LTER_EUROPE_EML,/metadata/idinfo/keywords/place/placekey


Now that we have our metadata raw data prepared and stored, we can prepare the collection's data for recommendation analytics and cross collection analytics.

Create a table with each record as a row of concept occurance counts. Each concept that occurs in the collection is a row.

In [25]:
FiledirectoryRAD=os.path.join('../data/',Organization)
FilenameRAD='/'+Collection+'_'+Dialect+'_RAD.csv'
FilePathRAD=FiledirectoryRAD+FilenameRAD
group_name = EvaluatedSimplifiedMetadataDF.groupby(['Record', 'Concept'], as_index=False)
occuranceMatrix=group_name.size().unstack()
occuranceMatrix=occuranceMatrix.fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
occuranceMatrix.to_csv(FilePathRAD, mode = 'w', index=False)
occuranceMatrix
FiledirectoryQuickE=os.path.join('../data/',Organization)
FilenameQuickE='/'+Collection+'_'+Dialect+'_QuickE.csv'
FilePathQuickE=FiledirectoryQuickE+FilenameQuickE
group_name = EvaluatedSimplifiedMetadataDF.groupby(['XPath', 'Record'], as_index=False)
QuickEdf=group_name.size().unstack().reset_index()
QuickEdf=QuickEdf.fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
QuickEdf.to_csv(FilePathQuickE, mode = 'w', index=False)
QuickEdf
FiledirectoryOccurance=os.path.join('../data/',Organization)
FilenameOccurance='/'+Collection+'_'+Dialect+'_Occurance.csv'
FilePathOccurance=FiledirectoryOccurance+FilenameOccurance
occuranceSum=occuranceMatrix.sum()
occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
CollectionName=FilenameOccurance.partition("/")[2].partition("_Occurance.csv")[0]
result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
result.insert(1, 'Collection', CollectionName)
result.insert(4, 'CollectionOccurance%', CollectionName)
result.insert(4, 'AverageOccurancePerRecord', CollectionName)
result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
NumberOfRecords = result.at[0, 'RecordCount']
result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)
result.at[0, 'ConceptCount'] = NumberOfRecords
result.at[0, 'Concept'] = 'Number of Records'
result['AverageOccurancePerRecord'] = result['ConceptCount']/NumberOfRecords
result['AverageOccurancePerRecord'] = result['AverageOccurancePerRecord'].astype(float)
result[["ConceptCount","RecordCount"]] = result[["ConceptCount","RecordCount"]].astype(int)
result['AverageOccurancePerRecord'] = pd.Series(["{0:.2f}".format(val) for val in result['AverageOccurancePerRecord']], index = result.index)
result.to_csv(FilePathOccurance, mode = 'w', index=False)
result

,Concept,Collection,ConceptCount,RecordCount,AverageOccurancePerRecord,CollectionOccurance%
0,Number of Records,LTER_EUROPE_EML,56,56,1.00,100.00%
1,Acknowledgement,LTER_EUROPE_EML,30,30,0.54,53.57%
2,Author,LTER_EUROPE_EML,155,55,2.77,98.21%
3,Author / Originator,LTER_EUROPE_EML,155,55,2.77,98.21%
4,Bounding Box,LTER_EUROPE_EML,40,40,0.71,71.43%
5,Contributor Name,LTER_EUROPE_EML,30,30,0.54,53.57%
6,Distribution Contact,LTER_EUROPE_EML,56,56,1.00,100.00%
7,Easternmost Longitude,LTER_EUROPE_EML,40,40,0.71,71.43%
8,Keyword,LTER_EUROPE_EML,433,48,7.73,85.71%
9,Keyword Type,LTER_EUROPE_EML,127,45,2.27,80.36%


Get RecTags csv into dataframe. use widget to select recommendation return that data series and split each string into the concept name and the recommendation profile. Add column of the concept occurance percentage. Add zeros for concepts that dont have a CO%. Use dialect contains to determine if the cell should be red or yellow. color cell green if CO% is 100%
count concepts for each recommendation profile and plot a line between them. Subtract the red cells from each profile and plot a line along the resultant totals. Label the x axis with profile names. y axis is concept count. make one solid one dashed line


count if greater than zero in certain columns

In [22]:
FiledirectoryQuickE=os.path.join('../data/',Organization)
FilenameQuickE='/'+Collection+'_'+Dialect+'_QuickE.csv'
FilePathQuickE=FiledirectoryQuickE+FilenameQuickE
group_name = EvaluatedSimplifiedMetadataDF.groupby(['XPath', 'Record'], as_index=False)
QuickEdf=group_name.size().unstack().reset_index()
QuickEdf=QuickEdf.fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
QuickEdf.to_csv(FilePathQuickE, mode = 'w', index=False)
QuickEdf

Record,XPath,ftp___ngee.ornl.gov_data_outgoing_metadata_ActiveLayer_and_Moisture_Measurements_IntensiveSite0_1.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Active_Layer_Soil_Carbon_and_Nitrogen_Mineralization_Barrow_Alaska_2012.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Addressing_numerical_challenges_in_introducing_a_reactive_transport_code_into_a_land_surface_model_A_biogeochemical_modeling_proof.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Airborne_Imagery_Collections_Barrow_2013.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_BEO_Tram_Spectral_Data_2014.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Barrow_Physiology_Data.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_CO2_CH4_flux_Air_temperature_Soil_temperature_and_Soil_moisture_Barrow_Alaska_2013.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_CO2_and_CH4_Production_and_CH4_Oxidation_in_Low_Temperature_Soil_Incubations_from_Flat_and_High_Centered_Polygons_Barrow_Alaska_2012.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_CO2_and_CH4_Production_in_Low_Temperature_Soil_Incubations_from_Low_and_High_Centered_Polygons_Barrow_Alaska_2012_2013.xml,...,ftp___ngee.ornl.gov_data_outgoing_metadata_Representativeness_based_Sampling_Network_Design_for_the_State_of_Alaska.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Snow_Depth_and_Density_at_End_of_Winter_for_NGEE_Areas_A_B_C_and_D_Barrow_Alaska_2012_2014.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Soil_Organic_Carbon_Degradation_Barrow_2013_2014.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Soil_Physicochemical_Characteristics_from_Ice_Wedge_Polygons_Barrow_Alaska_Ver_1.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Soil_Water_Characteristics_of_Cores_from_Low_and_High_Centered_Polygons_Barrow_Alaska_2012.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Soil_profiles_of_trace_gas_concentrations_and_stable_isotopes_temperature_and_moisture_Barrow_Alaska_2012_2013.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Soil_temperature_soil_moisture_and_thaw_depth_Barrow_Alaska_Ver_1.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Subsurface_Temperature_Moisture_Thermal_Conductivity_and_Heat_Flux_Barrow_Area_A_B_C_D.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Surface_Meteorology_Barrow_Area_A.xml,ftp___ngee.ornl.gov_data_outgoing_metadata_Time_lapse_photography_at_BEO_Barrow_Alaska_2014.xml
0,/metadata/Source,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,/metadata/dataqual,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,/metadata/dataqual/attracc/qattracc/attraccv,1,1,0,1,0,0,0,0,0,...,1,1,0,1,0,0,1,1,1,0
3,/metadata/dataqual/complete,1,1,0,1,0,0,0,0,0,...,1,1,0,1,0,0,1,1,1,1
4,/metadata/dataqual/lineage,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5,/metadata/dataqual/lineage/method/methcite/cit...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
6,/metadata/dataqual/lineage/method/methcite/cit...,0,0,0,0,0,0,0,0,1,...,1,0,0,1,1,1,0,0,0,0
7,/metadata/dataqual/lineage/method/methcite/cit...,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0
8,/metadata/dataqual/lineage/method/methcite/cit...,0,0,0,0,0,0,0,0,1,...,1,0,0,1,1,1,0,0,0,0
9,/metadata/dataqual/lineage/method/methdesc,1,1,0,0,0,0,1,1,1,...,1,1,0,1,1,1,1,1,1,0


In [24]:
FiledirectoryOccurance=os.path.join('../data/',Organization)
FilenameOccurance='/'+Collection+'_'+Dialect+'_Occurance.csv'
FilePathOccurance=FiledirectoryOccurance+FilenameOccurance
occuranceSum=occuranceMatrix.sum()
occuranceCount=occuranceMatrix[occuranceMatrix!=0].count()
CollectionName=FilenameOccurance.partition("/")[2].partition("_Occurance.csv")[0]
result = pd.concat([occuranceSum, occuranceCount], axis=1).reset_index()
result.insert(1, 'Collection', CollectionName)
result.insert(4, 'CollectionOccurance%', CollectionName)
result.insert(4, 'AverageOccurancePerRecord', CollectionName)
result.columns = ['Concept', 'Collection', 'ConceptCount', 'RecordCount', 'AverageOccurancePerRecord', 'CollectionOccurance%' ]
NumberOfRecords = result.at[0, 'RecordCount']
result['CollectionOccurance%'] = result['RecordCount']/NumberOfRecords
result['CollectionOccurance%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in result['CollectionOccurance%']], index = result.index)
result.at[0, 'ConceptCount'] = NumberOfRecords
result.at[0, 'Concept'] = 'Number of Records'
result['AverageOccurancePerRecord'] = result['ConceptCount']/NumberOfRecords
result['AverageOccurancePerRecord'] = result['AverageOccurancePerRecord'].astype(float)
result[["ConceptCount","RecordCount"]] = result[["ConceptCount","RecordCount"]].astype(int)
result['AverageOccurancePerRecord'] = pd.Series(["{0:.2f}".format(val) for val in result['AverageOccurancePerRecord']], index = result.index)
result.to_csv(FilePathOccurance, mode = 'w', index=False)
result

,Concept,Collection,ConceptCount,RecordCount,AverageOccurancePerRecord,CollectionOccurance%
0,Number of Records,LTER_EUROPE_EML,56,56,1.00,100.00%
1,Acknowledgement,LTER_EUROPE_EML,30,30,0.54,53.57%
2,Author,LTER_EUROPE_EML,155,55,2.77,98.21%
3,Author / Originator,LTER_EUROPE_EML,155,55,2.77,98.21%
4,Bounding Box,LTER_EUROPE_EML,40,40,0.71,71.43%
5,Contributor Name,LTER_EUROPE_EML,30,30,0.54,53.57%
6,Distribution Contact,LTER_EUROPE_EML,56,56,1.00,100.00%
7,Easternmost Longitude,LTER_EUROPE_EML,40,40,0.71,71.43%
8,Keyword,LTER_EUROPE_EML,433,48,7.73,85.71%
9,Keyword Type,LTER_EUROPE_EML,127,45,2.27,80.36%


### Select the notebook that prepares the data for different types of analysis

* [Create RAD Data](02RADdf.ipynb)
* [Cross Collection Comparisons](03CrossCollectionComparisons.ipynb)
* [Concept Content Consistency](04ConceptVerticals.ipynb)
* [Exploring Unknown Concepts](05ExploringUnknownConcepts.ipynb)